<a href="https://colab.research.google.com/github/germancruzram/Detector-de-grietas-CH/blob/main/Rigid_Pavement_Crack_Detector_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MODELO IA PARA DETECCION DE GRIETAS EN PAVIMENTO RIGIDO.**
**Creado por : German Cruz R. 🇳**

Mayo 2024.

# **PASO 1. INSTALANDO LAS LIBRERIAS**

In [ ]:
%%capture
!pip install super-gradients
!pip install imutils
!pip install gdown

In [ ]:
# Reiniciar sesión en Google Colab
import os
os._exit(00)


# **PASO 2. IMPORTANDO LOS INSTRUMENTOS**



In [ ]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
from IPython.display import clear_output
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from super_gradients.training import models
import os
import cv2
import pandas as pd
from PIL import Image, ExifTags
import numpy as np

The console stream is logged into /root/sg_logs/console.log


[2024-05-15 22:06:40] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-05-15 22:06:53] INFO - utils.py - NumExpr defaulting to 2 threads.


# **PASO 3. IMPORTANDO EL MODELO ENTRENADO PARA DETECCION DE GRIETAS**

In [ ]:
%%capture
url = 'https://drive.google.com/uc?id=1ozj7icywAbU8_qQmhuC-bqEYt_oRV7DD'

# MODELO IA ENTRENADO PARA DETECTAR GRIETAS
# CREADO POR GERMAN CRUZ R. MAYO 2024
output = '/content/crackCH_0524.pth'
!gdown {url} -O {output}

print("Archivo descargado y guardado en", output)

# **PASO 4. DEFINICION DE LAS CLASES**

In [ ]:
dataset_params = {
    'classes': ['grieta_lineal']

}

# **PASO 5. CARGANDO LA IA ENTRENADA**

In [ ]:
best_model = models.get('yolo_nas_s',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="/content/crackCH_0524.pth")



[2024-05-15 22:07:03] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/crackCH_0524.pth EMA checkpoint.


# **PASO 6. SUBIR LAS IMAGENES A PROCESAR EN LA CARPETA FOTOGRAFIA**

In [ ]:
import os
from google.colab import files

# Crear la carpeta "Fotografias"
folder_name = '/content/Fotografias'
os.makedirs(folder_name, exist_ok=True)
print(f'Carpeta "{folder_name}" creada exitosamente')

# Crear la carpeta "Detecciones" si no existe
detections_folder = '/content/Detecciones'
os.makedirs(detections_folder, exist_ok=True)
print(f'Carpeta "{detections_folder}" creada exitosamente')


# **PASO 7. EJECUTANDO LA DETECCIÓN**


In [ ]:
def get_lat_lon(exif):
    lat = lon = None
    if 'GPSInfo' in exif:
        gps_info = exif['GPSInfo']
        gps_latitude = gps_info.get(2)
        gps_latitude_ref = gps_info.get(1)
        gps_longitude = gps_info.get(4)
        gps_longitude_ref = gps_info.get(3)
        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = convert_to_degrees(gps_latitude)
            if gps_latitude_ref != 'N':
                lat = -lat
            lon = convert_to_degrees(gps_longitude)
            if gps_longitude_ref != 'E':
                lon = -lon
    return lat, lon

# Función para convertir los valores de latitud/longitud a grados decimales
def convert_to_degrees(value):
    d = float(value[0])
    m = float(value[1])
    s = float(value[2])
    return d + (m / 60.0) + (s / 3600.0)

In [ ]:
# Inicialización de la lista para almacenar los datos del Excel
data = []

# Ruta de la carpeta de imágenes
folder_path = '/content/Fotografias'
detections_folder = '/content/Detecciones'


# Procesar cada imagen en la carpeta "Fotografias"
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".jpg", ".png")):
        img_path = os.path.join(folder_path, filename)
        with Image.open(img_path) as img_pil:
            exif_data = img_pil.info.get('exif')
            exif = img_pil._getexif()
            if exif:
                exif = {ExifTags.TAGS[k]: v for k, v in exif.items() if k in ExifTags.TAGS}
                lat, lon = get_lat_lon(exif)
            else:
                lat, lon = None, None
            img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
            outputs = best_model.predict(img)
            detections = [{'confidence': confidence, 'bbox': bbox, 'class_id': label}
                          for bbox, confidence, label in zip(outputs.prediction.bboxes_xyxy, outputs.prediction.confidence, outputs.prediction.labels)
                          if confidence > 0.3]
            for det in detections:
                x_min, y_min, x_max, y_max = map(int, det['bbox'])
                cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (255, 0, 255), 2)
                label = f"{dataset_params['classes'][det['class_id']]}: {det['confidence']:.2f}"
                (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                cv2.rectangle(img, (x_min, y_min - text_height - 10), (x_min + text_width, y_min), (255, 0, 255), -1)
                cv2.putText(img, label, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            save_path = os.path.join(detections_folder, filename)
            img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if exif_data:
                img_pil.save(save_path, exif=exif_data)
            else:
                img_pil.save(save_path)
            data.append([filename, lat, lon, len(detections)])
        print(f"Processed {filename}")


# Crear el DataFrame y guardar en un archivo Excel
df = pd.DataFrame(data, columns=['Filename', 'Latitude', 'Longitude', 'Detections'])
excel_path = '/content/detections_info.xlsx'
df.to_excel(excel_path, index=False)
print("Proceso finalizado, las imágenes fueron procesadas y el archivo excel ha sido creado.")

# Verificar si hay un archivo CSV en la carpeta "Fotografias"
csv_file = None
for file in os.listdir(folder_path):
    if file.lower().endswith('.csv'):
        csv_file = os.path.join(folder_path, file)
        break

# Si se encuentra un archivo CSV, agregar la información del punto kilométrico
if csv_file:
    csv_data = pd.read_csv(csv_file)
    if 'Nombre_de_archivo_de_imagen' in csv_data.columns and 'Punto_kilométrico__km_' in csv_data.columns:
        detections_info = pd.read_excel(excel_path)
        merged_data = pd.merge(detections_info, csv_data[['Nombre_de_archivo_de_imagen', 'Punto_kilométrico__km_']],
                               left_on='Filename', right_on='Nombre_de_archivo_de_imagen', how='left')
        merged_data = merged_data.drop(columns=['Nombre_de_archivo_de_imagen'])
        merged_data = merged_data.rename(columns={'Punto_kilométrico__km_': 'PKM'})
        merged_data.to_excel(excel_path, index=False)
        print(f"Información de punto kilométrico agregada al archivo Excel en {excel_path}")

print("Proceso completado.")

In [ ]:
import shutil
# Ruta de la carpeta a comprimir
folder_to_zip = '/content/Detecciones'
# Ruta del archivo .zip resultante
zip_filename = '/content/Detecciones.zip'
# Comprimir la carpeta
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', folder_to_zip)
print(f'Carpeta "{folder_to_zip}" comprimida exitosamente como "{zip_filename}"')


# **BORRAR CONTENIDO DE LAS CARPETAS**

In [ ]:
import os
import shutil

# Función para eliminar todos los archivos en una carpeta
def delete_files_in_folder(folder_path):
    # Verificar si la carpeta existe
    if os.path.exists(folder_path):
        # Iterar sobre todos los archivos en la carpeta
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                # Eliminar archivos
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                # Eliminar directorios
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'No se pudo eliminar {file_path}. Razón: {e}')
        print(f'Todos los archivos en la carpeta "{folder_path}" han sido eliminados.')
    else:
        print(f'La carpeta "{folder_path}" no existe.')

# Rutas de las carpetas a limpiar
detections_folder = '/content/Detecciones'
fotografias_folder = '/content/Fotografias'

# Eliminar archivos en las carpetas especificadas
delete_files_in_folder(detections_folder)
delete_files_in_folder(fotografias_folder)


Todos los archivos en la carpeta "/content/Detecciones" han sido eliminados.
Todos los archivos en la carpeta "/content/Fotografias" han sido eliminados.
